<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/bi_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pronto transformers[torch] datasets evaluate \
&& pip install accelerate -U \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

In [3]:
import random
import pronto
import evaluate
import torch
import torch.nn.functional as F
import numpy as np
from copy import copy, deepcopy
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from datasets import Dataset, DatasetDict
from transformers import RobertaModel, AutoTokenizer, AutoModel, BioGptTokenizer, BioGptModel, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
from scipy.special import softmax

In [4]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")

In [5]:
# Get subsumptions from CSV file to a dictionary

def get_mappings_from_file(filename):
  mappings = {}
  with open(filename) as f:
    for line in f:
      source_id, target_id = line.strip().split(',')
      mappings[source_id] = target_id
  return mappings

In [6]:
equiv_doid2ncit = get_mappings_from_file("equiv_doid2ncit.csv")
equiv_ncit2doid = get_mappings_from_file("equiv_ncit2doid.csv")
subs_doid2ncit = get_mappings_from_file("subs_doid2ncit.csv")
subs_ncit2doid = get_mappings_from_file("subs_ncit2doid.csv")
neg_subs_doid2ncit = get_mappings_from_file("neg_subs_doid2ncit.csv")
neg_subs_ncit2doid = get_mappings_from_file("neg_subs_ncit2doid.csv")

# Convert relations to sentences

> Currently considering parents, children & siblings for conceptual reasons, but could also take 'n-hop' appraoch, e.g. 1-hop only with parents and children, or 2-hop to include grandparents, grandchildren and siblings.

> How do I incorporate the desired mapping for training? Should I incorporate both all this AND target info, or too much? Could be SELF + desired relatives instead, or SELF + PARENT + DESIRED PARENT, etc.

In [7]:
entity_markers = ["[SUB]", "[/SUB]", "[SUP]", "[/SUP]"]
sep_token = "[SEP]"
cls_token = "[CLS]"

In [8]:
# Create sentence from the given entity, containing its direct parents & siblings

def get_sentence(entity_id, onto):
  sub_in, sub_out, sup_in, sup_out = entity_markers

  subsumer = onto.get_term(entity_id)
  supersumers = list(subsumer.superclasses(distance=1, with_self=False))

  sentence = [sub_in, subsumer.name, sub_out]
  for supersumer in supersumers:
    sentence.extend([sup_in, supersumer.name, sup_out])

  return "".join(sentence)

In [9]:
# Create sentence from the given source entity, containing its mapping's parent & siblings

def get_combined_sentence(source_id, target_id, source_onto, target_onto):
  sub_in, sub_out, sup_in, sup_out = entity_markers

  if source_id not in source_onto.terms() or target_id not in target_onto.terms():
    return

  subsumer = source_onto.get_term(source_id)
  equivalent = target_onto.get_term(target_id)
  parents = list(equivalent.superclasses(distance=1, with_self=False))

  if len(parents) != 1:
    return
  supersumer = parents[0]

  #sentence = "".join([sub_in, subsumer.name, sub_out, sup_in, supersumer.name, sup_out])
  sentence = (sep_token + cls_token).join([subsumer.name, supersumer.name])
  return sentence

In [10]:
source_id = "DOID:0014667"
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
# tokenizer.add_tokens(entity_markers)

sentence = get_combined_sentence(source_id, subs_doid2ncit[source_id], doid, ncit)
tokenized = tokenizer(sentence)

print(sentence)
tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

disease of metabolism[SEP][CLS]Non-Neoplastic Disorder


['[CLS]',
 'disease',
 'of',
 'metabolism',
 '[SEP]',
 '[CLS]',
 'non',
 '-',
 'neoplastic',
 'disorder',
 '[SEP]']

# Train end-to-end BERT model

In [11]:
def generate_labelled_biencoder_samples(subs, negsubs, source_onto, target_onto):
  random.seed(3)
  source_samples = []
  target_samples = []
  labels = []

  pairs = list(subs.items()) + list(negsubs.items())
  zeros_and_ones = [0] * len(negsubs) + [1] * len(subs)
  labelled_pairs = list(zip(zeros_and_ones, pairs))
  random.shuffle(labelled_pairs)

  for label, (source_id, target_id) in tqdm(labelled_pairs):
    source_sentence = get_sentence(source_id, source_onto)
    target_sentence = get_sentence(target_id, target_onto)
    if source_sentence and target_sentence:
      source_samples.append(source_sentence)
      target_samples.append(target_sentence)
      labels.append(label)

  print()
  return source_samples, target_samples, labels

In [12]:
def collate_biencoder_data(X, y):
  source_X = [source_samples[i] for i in X]
  target_X = [target_samples[i] for i in X]
  X = np.array([val for pair in zip(source_X, target_X) for val in pair])
  y = np.array([val for label in y for val in (label, label)])
  return X, y

In [21]:
source_samples, target_samples, labels = generate_labelled_biencoder_samples(subs_doid2ncit, neg_subs_doid2ncit, doid, ncit)
print(f"Samples: {len(source_samples)}, {len(target_samples)}")
print(f"Labels: {len(labels)}")

100%|██████████| 3766/3766 [00:00<00:00, 6643.12it/s]


Samples: 3766, 3766
Labels: 3766


In [22]:
X = np.arange(len(source_samples))
y = np.array(labels)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=3)

print(X_train.shape, y_train.shape)

X_train, y_train = collate_biencoder_data(X_train, y_train)
X_val, y_val = collate_biencoder_data(X_val, y_val)
X_test, y_test = collate_biencoder_data(X_test, y_test)

print(X_train.shape, y_train.shape)

(2259,) (2259,)
(4518,) (4518,)


In [23]:
id2label = {0:"not_subsumption", 1:"subsumption"}
label2id = {label:ind for (ind, label) in id2label.items()}

dataset_train = Dataset.from_dict({'sample':X_train, 'label':y_train})
dataset_val = Dataset.from_dict({'sample':X_val, 'label':y_val})
dataset_test = Dataset.from_dict({'sample':X_test, 'label':y_test})
dataset = DatasetDict({'train':dataset_train,'val':dataset_val,'test':dataset_test})

In [16]:
def compute_metrics(eval_pred):
  accuracy = evaluate.load("accuracy")
  macro_metrics = [evaluate.load("precision"), evaluate.load('recall'), evaluate.load('f1')]

  predictions, references = eval_pred
  predictions = np.where(predictions.squeeze() >= 0, 1, 0)
  references = references[::2]

  metric_dict = accuracy.compute(predictions=predictions, references=references)
  for metric in macro_metrics:
    metric_dict.update(metric.compute(predictions=predictions, references=references, average='macro'))
  return metric_dict

In [17]:
class BiEncoderModel(torch.nn.Module):
  def __init__(self, model_name, num_labels, id2label=None, label2id=None, token_embeddings_size=None, hidden_layer=-1):
    super().__init__()
    self.source_model = AutoModel.from_pretrained(model_name)
    self.target_model = AutoModel.from_pretrained(model_name)
    if token_embeddings_size:
      self.source_model.resize_token_embeddings(token_embeddings_size)
      self.target_model.resize_token_embeddings(token_embeddings_size)
    self.source_model.config.pad_token_id = tokenizer.pad_token_id
    self.target_model.config.pad_token_id = tokenizer.pad_token_id

    self.num_labels = num_labels
    self.hidden_layer = hidden_layer

    self.linear = torch.nn.Linear(32, num_labels)
    self.dropout = torch.nn.Dropout(0.1)
    self.similarity = torch.nn.CosineSimilarity(dim=-1)
    self.loss = torch.nn.BCEWithLogitsLoss()

  @staticmethod
  def split_source_target(arr):
    source_arr, target_arr = None, None
    if arr is not None:
      source_arr, target_arr = arr[::2], arr[1::2]
    return source_arr, target_arr

  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

    source_ids, target_ids = BiEncoderModel.split_source_target(input_ids)
    source_attention, target_attention = BiEncoderModel.split_source_target(attention_mask)
    source_token_types, target_token_types = BiEncoderModel.split_source_target(token_type_ids)
    source_positions, target_positions = BiEncoderModel.split_source_target(position_ids)
    source_mask, target_mask = BiEncoderModel.split_source_target(head_mask)
    source_embeds, target_embeds = BiEncoderModel.split_source_target(inputs_embeds)

    source_outputs = self.source_model(
      source_ids,
      attention_mask=source_attention,
      token_type_ids=source_token_types,
      position_ids=source_positions,
      head_mask=source_mask,
      inputs_embeds=source_embeds,
    )

    target_outputs = self.target_model(
      target_ids,
      attention_mask=target_attention,
      token_type_ids=target_token_types,
      position_ids=target_positions,
      head_mask=target_mask,
      inputs_embeds=target_embeds,
    )

    pooled_source_outputs = self.dropout(source_outputs[1])
    pooled_target_outputs = self.dropout(target_outputs[1])
    logits = self.similarity(pooled_source_outputs, pooled_target_outputs)

    loss = None
    if labels is not None:
      clean_labels = labels[::2]
      loss = self.loss(logits.view(-1), clean_labels.view(-1).float())

    return SequenceClassifierOutput(loss=loss, logits=logits)

In [18]:
def prepare_model(dataset, id2label, label2id, entity_markers=entity_markers, learning_rate=1e-5, epochs=5, batch_size=16):
  random.seed(3)
  np.random.seed(3)
  torch.manual_seed(3)
  torch.cuda.manual_seed_all(3)
  torch.use_deterministic_algorithms(True)
  torch.backends.cudnn.deterministic = True

  pretrained = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

  tokenizer = AutoTokenizer.from_pretrained(pretrained)
  # tokenizer.add_tokens(entity_markers)
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})

  preprocess_tokenize = lambda data: tokenizer(data["sample"], padding="longest")
  tokenized_data = dataset.map(preprocess_tokenize, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
  model = BiEncoderModel(pretrained, num_labels=1, id2label=id2label, label2id=label2id, token_embeddings_size=len(tokenizer))

  training_args = TrainingArguments(
    output_dir="testing",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size*2,
    per_device_eval_batch_size=batch_size*2,
    num_train_epochs=epochs,
    logging_steps=1
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_data['train'],
      eval_dataset=tokenized_data['val'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  return trainer, tokenized_data

In [24]:
torch.cuda.empty_cache()
trainer, tokenized_data = prepare_model(dataset, id2label, label2id)
trainer.train()

Map:   0%|          | 0/4518 [00:00<?, ? examples/s]

Map:   0%|          | 0/1506 [00:00<?, ? examples/s]

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.564800,0.722377,0.531208,0.394441,0.488996,0.359356
2,0.629900,0.755436,0.471448,0.483850,0.485334,0.464626
3,0.680100,0.729850,0.487384,0.408821,0.455184,0.388346
4,0.561900,0.740031,0.487384,0.410693,0.455422,0.389897
5,0.758200,0.739627,0.495352,0.402404,0.460595,0.379822


TrainOutput(global_step=710, training_loss=0.5856489805688321, metrics={'train_runtime': 539.0656, 'train_samples_per_second': 41.906, 'train_steps_per_second': 1.317, 'total_flos': 0.0, 'train_loss': 0.5856489805688321, 'epoch': 5.0})

In [ ]:
predictions, label_ids, metrics = trainer.predict(tokenized_data['val'])
labels_predicted = [id2label[prediction] for prediction in np.argmax(predictions, axis=1)]
for metric in metrics.items():
  print(metric)